<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [131]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [72]:
seed = 7
np.random.seed(seed)
#Load data
url = 'https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv'
dataset = pd.read_csv(url)
print(dataset.shape)
dataset.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [156]:
features = dataset.columns[1:len(dataset.columns)-1]
target = 'Churn'
X = dataset[features]
y = dataset[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [157]:
sklearn.__version__

'0.22.1'

In [160]:
print(X_train.shape)
print(X_test.shape)

(5634, 6911)
(1409, 6911)


In [159]:
preprocessing = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False))
])
X_train = preprocessing.fit_transform(X_train.values)
X_test = preprocessing.transform(X_test.values)

In [138]:
y_train.shape

(5634,)

In [161]:
def keras_clf(hidden_layers=[64, 64, 64], dropout_rate=0, optimizer='adam', activation='relu', input_dim=12545, output_dim=1):
    '''paraphrased from http://ethen8181.github.io/machine-learning/keras/nn_keras_hyperparameter_tuning.html'''    
    model=Sequential()
    model.add(Dense(hidden_layers[0], input_dim=input_dim, activation=activation))
    for index, layers in enumerate(hidden_layers[1:]):
#         #Input layer + 1st Hidden Layer
        if index:
#             model.add(Dense(layers, input_dim=input_dim, activation=activation))
        #hidden layers
#         else:
            model.add(Dense(layers, activation=activation))
        #Dropout
        if dropout_rate:
            model.add(Dropout(rate=dropout_rate))
    #Ouptut layer
    if output_dim > 2:
        model.add(Dense(output_dim), activation='softmax')
        loss='categorical_crossentropy'
    else:
        model.add(Dense(output_dim, activation=activation))
        loss='binary_crossentropy'
    
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    return model

In [162]:
input_dim=X_train.shape[1]
output_dim=1

In [163]:
#Fixed parameters
model_keras = KerasClassifier(build_fn=keras_clf,
                              input_dim=input_dim,
                              output_dim=output_dim)

In [164]:
keras_fit_params = {
    
}

In [165]:
range_epochs = range(10, 30, 4)
range_layers = [[64, 64, 64, 64], [32, 32, 32, 32, 32], [100, 100, 100]]
range_batchsize = range(20, 20000, 2000)
range_optimizer = ['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam']
range_activations = ['relu', 'tanh', 'sigmoid']
range_dropouts = [n/100 for n in range(20, 50)]

In [166]:
keras_params_options = {
    'epochs': range_epochs,
    'dropout_rate': range_dropouts,
    'batch_size': range_batchsize,
    'optimizer': range_optimizer,
    'activation': range_activations,
#     'hidden_layers': range_layers
}

In [167]:
search = RandomizedSearchCV(model_keras,
                            param_distributions=keras_params_options,
                            cv=3,
                            n_iter=3,
                            n_jobs=1,
                            verbose=1)
best = search.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/10
3756/3756 [==============================] - 1s 203us/sample - loss: 2.5769 - acc: 0.6017
Epoch 2/10
3756/3756 [==============================] - 0s 101us/sample - loss: 2.3229 - acc: 0.6371
Epoch 3/10
3756/3756 [==============================] - 0s 103us/sample - loss: 2.2324 - acc: 0.6720
Epoch 4/10
3756/3756 [==============================] - 0s 100us/sample - loss: 2.0823 - acc: 0.6991
Epoch 5/10
3756/3756 [==============================] - 0s 102us/sample - loss: 1.9652 - acc: 0.7191
Epoch 6/10
3756/3756 [==============================] - 0s 102us/sample - loss: 1.9036 - acc: 0.7335
Epoch 7/10
3756/3756 [==============================] - 0s 101us/sample - loss: 1.8218 - acc: 0.7401
Epoch 8/10
3756/3756 [==============================] - 1s 162us/sample - loss: 1.6495 - acc: 0.7684
Epoch 9/10
3756/3756 [==============================] - 0s 119us/sample - loss: 1.5949 - acc: 0.7764
Epoch 10/10
1878/1878 [==============================] - 0s 210us/sample - loss: 2.3273 - a

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  3.0min finished


Epoch 1/22
5634/5634 [==============================] - 1s 189us/sample - loss: 0.8363 - acc: 0.3829
Epoch 2/22
5634/5634 [==============================] - 1s 123us/sample - loss: 0.7864 - acc: 0.4398
Epoch 3/22
5634/5634 [==============================] - 1s 112us/sample - loss: 0.7636 - acc: 0.4718
Epoch 4/22
5634/5634 [==============================] - 1s 116us/sample - loss: 0.7420 - acc: 0.4945
Epoch 5/22
5634/5634 [==============================] - 1s 113us/sample - loss: 0.7189 - acc: 0.5252
Epoch 6/22
5634/5634 [==============================] - 1s 114us/sample - loss: 0.7094 - acc: 0.5355
Epoch 7/22
5634/5634 [==============================] - 1s 106us/sample - loss: 0.6951 - acc: 0.5580
Epoch 8/22
5634/5634 [==============================] - 1s 108us/sample - loss: 0.6793 - acc: 0.5742
Epoch 9/22
5634/5634 [==============================] - 1s 103us/sample - loss: 0.6750 - acc: 0.5840
Epoch 10/22
5634/5634 [==============================] - 1s 106us/sample - loss: 0.6668 - a

In [169]:
best.best_estimator_.score(X_test, t_test)

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?